In [26]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from socket import socket

In [27]:
trash_pick_df = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')
trash_pick_df

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511,-86.633970,36.069130
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016,-86.741242,36.169482
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637


In [36]:
trash_pick_df = trash_pick_df.dropna(subset=('LATITUDE'))

In [37]:
trash_pick_df = trash_pick_df.dropna(subset=('LONGITUDE'))

In [38]:
missed_trash =trash_pick_df[trash_pick_df['Description'].str.contains('miss','Missed', na=False)]
missed_trash

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203.0,METRO,9208,21.0,1731459.367,666013.6012,-86.802988,36.160330
9,25331,11/1/2017,Trash - Curbside/Alley Missed Pickup,"Trash missed Tuesday again, ALLEY",1206 Ireland St,37208.0,METRO,9208,19.0,1733029.474,668762.6409,-86.797745,36.167916
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20219,267119,11/1/2019,Trash - Curbside/Alley Missed Pickup,church missed as wel,"1701 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1750024.587,669155.3125,-86.740180,36.169362
20220,267121,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed,"2709 Crestdale Dr, Nashville, TN 37214, United...",37214.0,RED RIVER,1502,15.0,1770240.199,676334.3993,-86.671860,36.189483
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637


In [53]:
red_picks = trash_pick_df[trash_pick_df['Trash Hauler'].str.contains('RED RIVER', na=False)]
red_picks

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20220,267121,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed,"2709 Crestdale Dr, Nashville, TN 37214, United...",37214.0,RED RIVER,1502,15.0,1770240.199,676334.3993,-86.671860,36.189483
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511,-86.633970,36.069130
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637


In [57]:
red_picks_miss = red_picks.duplicated(subset = ['Incident Address', 'Trash Hauler'], keep = 'first')
red_picks_miss

0        False
1        False
2        False
4        False
5        False
         ...  
20220    False
20221     True
20223     True
20224     True
20225    False
Length: 14390, dtype: bool

In [65]:
red_picks_missed = red_picks_miss.count()*200
red_picks_missed

2878000

In [54]:
red_picks.nunique()

Request Number      14390
Date Opened           650
Request                 4
Description          8988
Incident Address    10115
Zip Code               22
Trash Hauler            1
Trash Route           126
Council District       32
State Plan X        10160
State Plan Y        10147
LONGITUDE           10187
LATITUDE            10065
dtype: int64

In [39]:
trash_complain  =trash_pick_df[trash_pick_df['Request'].str.contains('Trash Collection Complaint', na=False)]
trash_complain

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203.0,METRO,9208,21.0,1731459.367,666013.6012,-86.802988,36.160330
11,25350,11/1/2017,Trash Collection Complaint,Missed- 4th week in a row.,3210 Whites Creek Pike,37207.0,RED RIVER,3201,3.0,1726432.966,691919.2003,-86.820750,36.231374
13,25366,11/1/2017,Trash Collection Complaint,trash truck took cart \n3 guys on truck did no...,2200 osage st,37208.0,METRO,9201,21.0,1728644.757,672074.0991,-86.812692,36.176913
16,25393,11/1/2017,Trash Collection Complaint,trash carts all tossed in road after pick up,2720 riverbend dr,37214.0,RED RIVER,1501,15.0,1762455.924,693225.6657,-86.698647,36.235731
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20183,266732,11/1/2019,Trash Collection Complaint,Caller stated her trash misses her trash all t...,"2114 Vine Hill Rd, Nashville, TN 37204, United...",37204.0,METRO,9406,17.0,1741197.382,655129.2365,-86.769711,36.130646
20197,266844,11/1/2019,Trash Collection Complaint,RESIDENT CAME HOME YESTERDAY TO FIND THAT THE ...,"2812 Vaulx Ln, Nashville, Tennessee, 37204",37204.0,METRO,9404,17.0,1738157.855,652143.5792,-86.779923,36.122379
20200,266891,11/1/2019,Trash Collection Complaint,"garbage truck is cutting the corner too close,...","1997 Nashboro Blvd, Nashville, Tennessee, 37217",37217.0,RED RIVER,2505,29.0,1781292.396,639805.4380,-86.633606,36.089342
20211,267014,11/1/2019,Trash Collection Complaint,MANAGER SAYS HE WITNESSED THE TRASH TRUCK COME...,"109 2nd Ave N, Nashville, TN 37201, United States",37201.0,NaN,NaN,19.0,1739543.400,666600.8010,-86.775620,36.162122


In [ ]:
print('Trash 

In [40]:
nash_map = folium.Map(location = [36.1612, -86.7775], zoom_start =12)
nash_map

In [41]:
trash_complain = trash_complain.dropna(subset =['LONGITUDE'])

In [42]:
trash_complain = trash_complain.dropna(subset=('LATITUDE'))

In [43]:
row = trash_complain.iloc[1]
row

Request Number                                             25327
Date Opened                                            11/1/2017
Request                               Trash Collection Complaint
Description         Trash out on time, miss again Tuesday. ALLEY
Incident Address                            1816 Jo Johnston Ave
Zip Code                                                 37203.0
Trash Hauler                                               METRO
Trash Route                                                 9208
Council District                                            21.0
State Plan X                                         1731459.367
State Plan Y                                         666013.6012
LONGITUDE                                             -86.802988
LATITUDE                                                36.16033
Name: 7, dtype: object

In [47]:
nash_map = folium.Map(location = [36.1612, -86.7775], zoom_start =12)

for index, row in trash_complain.iterrows():
    loc = [row['LATITUDE'], row['LONGITUDE']]
    pop = str(row['Description'])
    folium.Marker(location = loc, pop = popup).add_to(nash_map)
    
nash_map

In [49]:
metro =trash_pick_df[trash_pick_df['Trash Hauler'].str.contains('METRO', na=False)]
metro

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203.0,METRO,9208,21.0,1731459.367,666013.6012,-86.802988,36.160330
9,25331,11/1/2017,Trash - Curbside/Alley Missed Pickup,"Trash missed Tuesday again, ALLEY",1206 Ireland St,37208.0,METRO,9208,19.0,1733029.474,668762.6409,-86.797745,36.167916
12,25359,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed. Does not have access to alley.,830 Meridian St,37207.0,METRO,9207,5.0,1741744.712,675328.6661,-86.768395,36.186144
13,25366,11/1/2017,Trash Collection Complaint,trash truck took cart \n3 guys on truck did no...,2200 osage st,37208.0,METRO,9201,21.0,1728644.757,672074.0991,-86.812692,36.176913
22,25471,11/1/2017,Trash - Curbside/Alley Missed Pickup,"Alley trash miss Tuesday, also others as well,...",1904 MeHarry Blvd,37208.0,METRO,9208,21.0,1730048.071,668927.1905,-86.807850,36.168301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20212,267023,11/1/2019,Trash - Curbside/Alley Missed Pickup,Trash/missed Thursday {carts sit's on the Arch...,"1098 12th Ave S, Nashville, TN 37203, United S...",37203.0,METRO,9407,17.0,1736440.901,660570.3200,-86.785965,36.145489
20213,267035,11/1/2019,Trash - Curbside/Alley Missed Pickup,"constantly missed, please pick up","1400 Dickerson Pike, Nashville, TN 37207, Unit...",37207.0,METRO,9204,5.0,1740796.199,679412.7988,-86.771718,36.197342
20216,267089,11/1/2019,Trash - Curbside/Alley Missed Pickup,The entire street/missed/curb Thursday,"1008 Waverly Ave, Nashville, TN 37203, United ...",37203.0,METRO,9407,17.0,1736525.000,657016.0014,-86.785584,36.135727
20219,267119,11/1/2019,Trash - Curbside/Alley Missed Pickup,church missed as wel,"1701 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1750024.587,669155.3125,-86.740180,36.169362


In [64]:
print(trash_complain['Description'])

5        left trash cart in middle of driveway instead ...
7             Trash out on time, miss again Tuesday. ALLEY
11                              Missed- 4th week in a row.
13       trash truck took cart \n3 guys on truck did no...
16            trash carts all tossed in road after pick up
                               ...                        
20183    Caller stated her trash misses her trash all t...
20197    RESIDENT CAME HOME YESTERDAY TO FIND THAT THE ...
20200    garbage truck is cutting the corner too close,...
20211    MANAGER SAYS HE WITNESSED THE TRASH TRUCK COME...
20215    trash truck broke a limb out of her tree and l...
Name: Description, Length: 2311, dtype: object
